# Advanced joining

## What can go wrong?

|             | Missing             | Duplicate             |
|-------------|---------------------|-----------------------|
| Key values  | Missing key values  | Duplicate key values  |
| Key columns | Missing key columns | Duplicate key columns |


### Missing key values

Since there's one key value missing in namesNA dataset, it needs some treatment before putting two datasets together.

* namesNA

|   | name | band    |
|---|------|---------|
| 1 | Mick | Stones  |
| 2 | NA   | Beatles |
| 3 | Paul | Beatles |

* plays

|   | name  | plays  |
|---|-------|--------|
| 1 | John  | Guitar |
| 2 | Paul  | Bass   |
| 3 | Keith | Guitar |

```
# Remove such rows from the data using the dplyr filter command
namesNA %>%
    filter(!is.na(name)) %>%
    left_join(plays, by = "name")
```

### Missing key columns

There's no common column name in two datasets, so it cannot be put together. But if you look closely, the rownames has the same information in name column of plays dataset

* noNames

|      | surname   | band    |
|------|-----------|---------|
| Mick | Jagger    | <NA>    |
| John | Lennon    | Beatles |
| Paul | McCartney | Beatles |

* plays

|   | name  | plays  |
|---|-------|--------|
| 1 | John  | Guitar |
| 2 | Paul  | Bass   |
| 3 | Keith | Guitar |

```
# Change rownames to column using a function in tibble library
library(tibble)
rownames_to_column(noNames, var = "name")
```

### Duplicate key values

In some cases, duplicate keys are a natural feature of the data.

* showNames

|   | name               | composer             |
|---|--------------------|----------------------|
| 1 | The Sound of Music | Richard Rogers       |
| 2 | The Sound of Music | Oscar Hammerstein II |
| 3 | The King and I     | Richard Rogers       |
| 4 | The King and I     | Oscar Hammerstein II |

### Spot the key

R's data frames can store important information in the row.names attribute. This is not a tidy way to store data, but it does happen quite commonly. If the primary key of your dataset is stored in row.names, you will have trouble joining it to other datasets.

For example, stage_songs contains information about songs that appear in musicals. However, it stores the primary key (song name) in the row.names attribute. As a result, you cannot access the key with a join function.

One way to remedy this problem is to use the function rownames_to_column() from the tibble package. rownames_to_column() returns a copy of a dataset with the row names added to the data as a column.

* Load the tibble package.
* Use rownames_to_column() to add the row names of stage_songs to the data frame as a column named song.
* Left join stage_writers to stage_songs.

In [2]:
stage_songs <- data.frame(musical = c("Into the Woods", "West Side Story",
                                        "Cats", "Phantom of the Opera"),
                            year = c(1986, 1957, 1981, 1986))
rownames(stage_songs) <- c("Children Will Listen", "Maria", "Memory", "The Music of the Night")

stage_writers <- data.frame(song = c("Children Will Listen", "Maria",
                                     "Memory", "The Music of the Night"),
                            composer = c("Stephen Sondheim", "Leonard Bernstein",
                                        "Andrew Lloyd Webber", "Andrew Lloyd Webber"))

library(dplyr)

# Load the tibble package
library(tibble)

stage_songs %>% 
  # Add row names as a column named song
  rownames_to_column(var = "song") %>% 
  # Left join stage_writers to stage_songs
  left_join(stage_writers, by = c("song"))

Warning message:
"package 'dplyr' was built under R version 3.4.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'tibble' was built under R version 3.4.3"Warning message:
"Column `song` joining character vector and factor, coercing into character vector"

song,musical,year,composer
Children Will Listen,Into the Woods,1986,Stephen Sondheim
Maria,West Side Story,1957,Leonard Bernstein
Memory,Cats,1981,Andrew Lloyd Webber
The Music of the Night,Phantom of the Opera,1986,Andrew Lloyd Webber


### Missing keys

Julie Andrews is a famous singer who performed in The Sound of Music, which is documented in the singers data frame. The two_songs data frame records a couple of songs that she sang during her career.

Unfortunately, when you join the datasets together, something odd happens. According to the result, who sings A Spoonful of Sugar? Why does this happen?

You can avoid this outcome by removing rows that contain NAs in the keys before joining.

Take a look at singers and two_songs in the console.

* Examine the result of joining singers to two_songs.
* Add a filter() call to the pipe to remove NA's from the key column of two_songs before calling inner_join().

In [3]:
two_songs_df <- data.frame(song = c("Do-Re-Mi", "A Spoonful of Sugar"),
                          movie = c("The Sound of Music", NA))
two_songs <- as.tibble(two_songs_df)

singers_df <- data.frame(movie = c(NA, "The Sound of Music"),
                        singer = c("Arnorld Schwarzenegger", "Julie Andrews"))
singers <- as.tibble(singers_df)

# Examine the result of joining singers to two_songs
two_songs %>% inner_join(singers, by = "movie")

# Remove NA's from key before joining
two_songs %>% 
  filter(!is.na(movie)) %>% 
  inner_join(singers, by = "movie")

song,movie,singer
Do-Re-Mi,The Sound of Music,Julie Andrews
A Spoonful of Sugar,NA,Arnorld Schwarzenegger


song,movie,singer
Do-Re-Mi,The Sound of Music,Julie Andrews


## Defining the keys

* names

|   | name | band    |
|---|------|---------|
| 1 | Mick | Stones  |
| 2 | John | Beatles |
| 3 | Paul | Beatles |


* plays

|   | name  | plays  |
|---|-------|--------|
| 1 | John  | Guitar |
| 2 | Paul  | Bass   |
| 3 | Keith | Guitar |

```
# Define the keys explicitly
left_join(names, plays, by = "name")

# If you omit the by argument, the function will automatically find the common key and use it.

left_join(names, plays)
```

### Mismatched key names

* members

|   | members | band    |
|---|---------|---------|
| 1 | Mick    | Stones  |
| 2 | John    | Beatles |
| 3 | Paul    | Beatles |


* plays

|   | name  | plays  |
|---|-------|--------|
| 1 | John  | Guitar |
| 2 | Paul  | Bass   |
| 3 | Keith | Guitar |

```
# When the key column names are different,
# tell the function that they are the same
left_join(members, plays, by = c("member" = "name"))
```

### Conflicting names


* playsWith

|   | members | plays   |
|---|---------|---------|
| 1 | Mick    | Stones  |
| 2 | John    | Beatles |
| 3 | Paul    | Beatles |


* plays

|   | name  | plays  |
|---|-------|--------|
| 1 | John  | Guitar |
| 2 | Paul  | Bass   |
| 3 | Keith | Guitar |

```
# dplyr will add suffixes to column names
left_join(playsWith, plays, by = "name", suffix = c("1", "2"))
```

### Which keys?

Frank Sinatra and Bing Crosby each crossed the boundary from music to acting, singing famous songs in well-known movies. movie_years and movie_songs describe some of their work.

Examine movie_years and movie_songs to determine which keys dplyr would use to join the datasets if you did not set the by argument. In this case, it would not matter which type of join you use, or in which order you call the datasets.

** movie & name **

### A subset of keys

Often the same column name will be used by two datasets to refer to different things. For example, the data frame movie_studios uses name to refer to the name of a movie studio. movie_years uses name to refer to the name of an actor.

You could join these datasets (they describe the same movies), but you wouldn't want to use the name column to do so!

dplyr will ignore duplicate column names if you set the by argument and do not include the duplicated name in the argument. When you do this, dplyr will treat the columns in the normal fashion, but it will add .x and .y to the duplicated names to help you tell the columns apart.

* Left join movie_studios to movie_years by the movie column. What happens to the duplicated column names?
* Use rename() to rename the name column of movie_years to artist, and the name column of movie_studios to studio. As an example, rename(data, new_name = old_name) renames old_name to new_name in data.

In [ ]:
movie_years %>% 
  # Left join movie_studios to movie_years
  left_join(movie_studios, by = "movie") %>%
  # Rename the columns: artist and studio
  rename(artist = name.x, studio = name.y)

### Mis-matched key names

Just as the same name can refer to different things in different datasets, different names can refer to the same thing. For example, elvis_movies and elvis_songs both describe movies starring Elvis Presley, but each uses a different column name to describe the name of the movie.

This type of inconsistency can be frustrating when you wish to join data based on the inconsistently named variable.

To make the join, set by to a named vector. The names of the vector will refer to column names in the primary dataset (x). The values of the vector will correspond to the column names in the secondary dataset (y), e.g.

x %>% left_join(y, by = c("x.name1" = "y.name2"))
dplyr will make the join and retain the names in the primary dataset.


* Identify the column in elvis_songs that corresponds to a column in elvis_movies.
* Left join elvis_songs to elvis_movies by this column.
* Use rename() to give the result the column names movie, year, and song.

In [ ]:
# Identify the key column
elvis_songs
elvis_movies

elvis_movies %>% 
  # Left join elvis_songs to elvis_movies by this column
  left_join(elvis_songs, by = c("name" = "movie"))%>%
  # Rename columns
  rename(movie = name, song = name.y)

### More mismatched names

Now, you'll practice what you learned in the last exercise.

movie_years and movie_directors both describe movies that feature Frank Sinatra and Bing Crosby. However, each data frame uses different column names to refer to the key variables that connect the datasets.

Can you join the data?


* Identify the key columns that connect movie_directors to movie_years.
* Left join movie_directors to movie_years.
* Use select() to arrange the columns in the following order with the following names: year, movie, artist, director, studio.


In [ ]:
# Identify the key columns
movie_directors
movie_years

movie_years %>% 
  # Left join movie_directors to movie_years
  left_join(movie_directors, by = c("movie" = "name")) %>% 
  # Arrange the columns using select()
  select(year, movie, artist = name, director, studio)

## Joining multiple tables

### purrr R package

* Applies functions in efficient ways
* reduce()
* Works well with dplyr

### Installing purrr

```
install.packages("purrr")
library(purrr)
```

### reduce()

* surnames

| name  | surname |
|-------|---------|
| Mick  | Jagger  |
| John  | Lennon  |
| Ringo | Starr   |

* names

| name  | band    |
|-------|---------|
| Mick  | Stones  |
| John  | Beatles |
| Paul  | Beatles |

* plays

| name  | plays   |
|-------|---------|
| Mick  | Guitar  |
| John  | Bass    |
| Keith | Guitar  |

```
tables <- list(surnames, names, plays)
reduce(tables, left_join, by = "name")

# reduce() does the equivalent of the following pipeline
surnames %>%
    left_join(names, by = "name") %>%
    left_join(plays, by = "name")
```

### Join multiple tables

purrr's reduce() function is very useful for joining together multiple datasets. Let's practice using it with three new datasets.

supergroups lists the members of several "supergroup" bands, which are bands made from the members of other famous bands. more_bands lists the original band membership of many musicians, including those in supergroups. And more_artists lists the instruments that various artists played.

Can you join these datasets into a single, coherent dataset?


* Load the purrr library.
* Place supergroups, more_bands, and more_artists into a list, in that order.
* Use reduce() to join together supergroups, more_bands, and more_artists into a dataset that contains all available information about each artist in supergroups (and no others).

In [ ]:
# Load the purrr library
library(purrr)

# Place supergroups, more_bands, and more_artists into a list
list(supergroups, more_bands, more_artists) %>% 
  # Use reduce to join together the contents of the list
  reduce(left_join, by = c("first", "last"))

### Filter multiple tables
Nice work! The job of reduce() is to apply a function in an iterative fashion to many datasets. As a result, reduce() works well with all of the dplyr join functions.

For example, you can use reduce() to filter observations with a filtering join.


* Use reduce() to return just the rows of more_artists that appear in all three datasets.

In [ ]:
list(more_artists, more_bands, supergroups) %>% 
  # Return rows of more_artists in all three datasets
  reduce(semi_join, by = c("first", "last"))

## Other implementations

### merge()

```
merge(names, plays, by = "name", ...)

# left_join(names, plays, by = "name")
merge(names, plays, by = "name", all.x = TRUE, all.y = FALSE)

# right_join(names, plays, by = "name")
merge(names, plays, by = "name", all.x = FALSE, all.y = TRUE)

# inner_join(names, plays, by = "name") is a default in merge
merge(names, plays, by = "name", all = FALSE)

# full_join(names, plays, by = "name")
merge(names, plays, by = "name", all = TRUE)
```

### Comparing with SQL

| R            | SQL                                       |
|--------------|-------------------------------------------|
| inner_join() | SELECT * FROM x JOIN y ON x.a = y.a       |
| left_join()  | SELECT * FROM x LEFT JOIN y ON x.a = y.a  | 
| right_join() | SELECT * FROM x RIGHT JOIN y ON x.a = y.a |
| full_join()  | SELECT * FROM x FULL JOIN y ON x.a = y.a  |
| semi_join()  | SELECT * FROM x WHERE EXISTS (SELECT l FROM y WHERE x.a = y.a) |
| anit_join()  | SELECT * FROM x WHERE NOT EXISTS (SELECT l FROM y WHERE x.a = y.a) |

### dplyr SQL connections

| Function       | DBMS           |
|----------------|----------------|
| src_sqlite()   | SQLite         |
| src_mysql()    | MySQL, MariaDB |
| src_postgres() | PostgresSQL    |

> install.packages("DBI") # Install DBI package to use them

### Base R

dplyr joins are also similar to the merge() function in base R.

For example, the code in the editor uses merge() and arrange() to perform a left_join() on bands and artists.


* Alter the code to perform the join with a dplyr function.
* Remove arrange() if your solution does not need it.

In [ ]:
# Alter the code to perform the join with a dplyr function
merge(bands, artists, by = c("first", "last"), all.x = TRUE) %>%
  arrange(band)

bands %>%
  left_join(artists, by = c("first", "last")) %>%
  arrange()